In [1]:
import re
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup as bs
import time
import sqlite3
from urllib.parse import quote_plus
import yaml
from datetime import datetime
import os
from KBDownloader import search_swedish_newspapers, fetch_newspaper_data, save_checkpoint, load_checkpoint

# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')

# Load the YAML configuration file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Assign variables from the YAML configuration
venue_list = config['venue_list']
start_year = config['start_year']
years_to_crawl = config['years_to_crawl']
newspaper = config['newspaper']
db_path = config['db_path']

# Define the newspaper collection IDs
NEWSPAPER_COLLECTION_IDS = {
    'Dagens nyheter': 'https://libris.kb.se/m5z2w4lz3m2zxpk#it',
    'Svenska Dagbladet': 'https://libris.kb.se/2ldhmx8d4mcrlq9#it',
    'Aftonbladet': 'https://libris.kb.se/dwpgqn5q03ft91j#it',
    'Dagligt Allehanda': 'https://libris.kb.se/9tmqzv3m32xfzcz#it'
}

# Get the correct collection ID for the specified newspaper
collection_id = NEWSPAPER_COLLECTION_IDS.get(newspaper)
if not collection_id:
    raise ValueError(f"Invalid newspaper name: {newspaper}")

# Ensure the database file exists
if not os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    conn.close()

# Load checkpoint if it exists
checkpoint = load_checkpoint()
start_year = checkpoint['year'] if checkpoint else start_year
start_half = checkpoint['half'] if checkpoint else 0
start_index = checkpoint['index'] if checkpoint else 0

# Print out all the settings from the YAML configuration file
print("Configuration Settings:")
for key, value in config.items():
    print(f"{key}: {value}")
print(f"Collection ID: {collection_id}")

# Load the venue list
df = pd.read_excel(venue_list)

Configuration Settings:
venue_list: Datasets/test_venues_just_royal_opera.xlsx
start_year: 1908
years_to_crawl: 1
newspaper: Dagens nyheter
prompt_filepath: oldtimey_touringbot_prompt_for_deployment.txt
db_path: Datasets/12.07_test.db
llm_model: gpt-3.5-turbo
max_tokens: 1000
Collection ID: https://libris.kb.se/m5z2w4lz3m2zxpk#it


In [2]:
# Main loop
for year in range(start_year, start_year + years_to_crawl):
    for half in range(2):
        if year == start_year and half < start_half:
            continue
        
        if half == 0:
            from_date = datetime(year, 1, 1)
            to_date = datetime(year, 6, 30)
        else:
            from_date = datetime(year, 7, 1)
            to_date = datetime(year, 12, 31)

        for index, row in df.iloc[start_index:].iterrows():
            query = row['Lokal']
            safe_query = "".join([c if c.isalnum() else "_" for c in query])

            
            try:
                result = fetch_newspaper_data(
                    query=query,
                    from_date=from_date.strftime('%Y-%m-%d'),
                    to_date=to_date.strftime('%Y-%m-%d'),
                    newspaper=collection_id,
                    config=config,
                    db_path=db_path
                )
                
                if result.get('success'):
                    print(f"Processed query '{query}' successfully.")
                else:
                    print(f"Failed to process query '{query}': {result.get('message')}")
                
                # Save checkpoint after each query, successful or not
                save_checkpoint(year, half, index + 1)
                
            except Exception as e:
                print(f"Error processing query '{query}': {str(e)}")
                save_checkpoint(year, half, index)
                raise  # Re-raise the exception to stop the script

        print(f"Waiting so KB does not get mad. Currently at {from_date} to {to_date}")
        time.sleep(3) # in seconds
        start_index = 0  # Reset start_index for the next half-year

    start_half = 0  # Reset start_half for the next year

print("All queries processed for all specified years.")

2024-07-12 18:32:56,245 - INFO - Starting fetch_newspaper_data for query: Kungliga opera, dates: 1908-01-01 to 1908-06-30
2024-07-12 18:32:56,501 - INFO - Search results received. Hits: 20
2024-07-12 18:32:56,502 - INFO - Extracted 20 URLs from search results
2024-07-12 18:32:56,504 - INFO - Table 'newspaper_data' created or already exists
2024-07-12 18:32:56,504 - INFO - Processing URL: https://data.kb.se/dark-3693840/part/1/page/2
2024-07-12 18:32:56,660 - INFO - Extracted 1 XML URLs
2024-07-12 18:32:56,984 - INFO - Fetched XML content for 1 pages
2024-07-12 18:32:57,148 - INFO - Committed changes for URL: https://data.kb.se/dark-3693840/part/1/page/2
2024-07-12 18:32:57,149 - INFO - Processing URL: https://data.kb.se/dark-3693618/part/1/page/1
2024-07-12 18:32:57,299 - INFO - Extracted 1 XML URLs
2024-07-12 18:32:57,649 - INFO - Fetched XML content for 1 pages
2024-07-12 18:32:57,857 - INFO - Committed changes for URL: https://data.kb.se/dark-3693618/part/1/page/1
2024-07-12 18:32:5

Processed query 'Kungliga opera' successfully.
Checkpoint saved: Year 1908, Half 0, Index 1
Waiting so KB does not get mad. Currently at 1908-01-01 00:00:00 to 1908-06-30 00:00:00


2024-07-12 18:33:14,445 - INFO - Starting fetch_newspaper_data for query: Kungliga opera, dates: 1908-07-01 to 1908-12-31
2024-07-12 18:33:14,722 - INFO - Search results received. Hits: 20
2024-07-12 18:33:14,723 - INFO - Extracted 20 URLs from search results
2024-07-12 18:33:14,725 - INFO - Table 'newspaper_data' created or already exists
2024-07-12 18:33:14,726 - INFO - Processing URL: https://data.kb.se/dark-3694203/part/1/page/2
2024-07-12 18:33:14,884 - INFO - Extracted 1 XML URLs
2024-07-12 18:33:15,151 - INFO - Fetched XML content for 1 pages
2024-07-12 18:33:15,422 - INFO - Committed changes for URL: https://data.kb.se/dark-3694203/part/1/page/2
2024-07-12 18:33:15,423 - INFO - Processing URL: https://data.kb.se/dark-3694116/part/1/page/3
2024-07-12 18:33:15,576 - INFO - Extracted 1 XML URLs
2024-07-12 18:33:15,940 - INFO - Fetched XML content for 1 pages
2024-07-12 18:33:16,135 - INFO - Committed changes for URL: https://data.kb.se/dark-3694116/part/1/page/3
2024-07-12 18:33:1

KeyboardInterrupt: 

# View SQL Database

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# SQL query to select all columns except 'Raw API Result'
query = """
SELECT Date, [Package ID], Part, Page, [ComposedBlock Content], [Full Prompt]
FROM newspaper_data
"""

# Read the query results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the first few rows of the DataFrame
print(df.head())

# Display basic information about the DataFrame
print(df.info())

# Optional: If you want to see all rows, you can use:
# pd.set_option('display.max_rows', None)
# print(df)

# Optional: If you want to save this to a CSV file for further analysis:
# df.to_csv('newspaper_data_summary.csv', index=False)